### Unbiased Variance Estimation with Fixed Initial Elements and Conditional Normal Distribution

This section presents the formula for calculating the unbiased variance estimate for an ARMA model, considering fixed initial elements and a conditional normal distribution.

#### Notations:

- $ p $ — the order of the AR model.
- $ q $ — the order of the MA model.
- $ \max(p, q) $ — the number of initial elements that are fixed.
- $ N $ — the total number of observations.
- $ k = p + q $ — the total number of model parameters (AR and MA parameters).

#### Variance Formula:

$$
\hat{\sigma}^2 = \frac{1}{N - \max(p, q) - k} \sum_{t=\max(p, q) + 1}^{N} \epsilon_t^2
$$


In [1]:
import numpy as np
import sympy as sp

class AnalyticalAR:
    def __init__(self, order):
        self.order = order  # The order of the AR model (AR(p))
        self.phi = sp.symbols(f'phi1:{self.order + 1}')  # Creating symbolic variables for AR model parameters
        self.sigma2 = sp.symbols('sigma2')  # Noise variance

    def fit(self, y):
        """
        Method to analytically fit the AR model parameters using the log-likelihood function.

        Parameters:
        y : numpy array
            Time series data.
        """
        N = len(y)

        # Creating a symbolic array for y
        y_sym = sp.IndexedBase('y')
        i = sp.symbols('i')

        # Defining the expression for the noise variance sigma^2
        residual = y_sym[i] - sum(self.phi[j] * y_sym[i-j-1] for j in range(self.order))
        sigma2 = (1 / (N - 2 * self.order)) * sp.Sum(residual**2, (i, self.order + 1, N)).doit()

        # Creating the log-likelihood expression
        log_likelihood = -(N - self.order)/2 * sp.log(2 * np.pi * sigma2)

        # Differentiating the log-likelihood with respect to each phi parameter
        diff_log_likelihood = [sp.diff(log_likelihood, self.phi[j]) for j in range(self.order)]

        # Solving the equations to find the optimal phi values
        phi_solutions = sp.solve(diff_log_likelihood, self.phi)

        # Substituting the actual values of y into the phi solutions
        y_values = {y_sym[i]: y[i-1] for i in range(1, N+1)}
        self.phi_values = {self.phi[j]: phi_solutions[self.phi[j]].subs(y_values) for j in range(self.order)}

    def predict(self, y, steps=1):
        """
        Method to predict future values of the time series.

        Parameters:
        y : numpy array
            Time series data.
        steps : int
            Number of steps to predict.

        Returns:
        predictions : numpy array
            Predicted values.
        """
        predictions = []
        for _ in range(steps):
            pred = sum(self.phi_values[self.phi[j]] * y[-j-1] for j in range(self.order))
            predictions.append(pred)
            y = np.append(y, pred)

        return np.array(predictions)

# Example usage of the class
y = np.array([1, 2, 2, 5, 8, 14])  # The time series

# Creating and fitting an AR(p) model
ar_model = AnalyticalAR(order=2)
ar_model.fit(y)

# Predicting the next 3 steps
predictions = ar_model.predict(y, steps=3)
print("Predicted values:", predictions)

Predicted values: [23.8518518518518 41.1165980795610 70.5128283290149]
